<a href="https://colab.research.google.com/github/DarvinX/InNav/blob/master/monte_carlo_1d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from numpy import random
from math import *

In [2]:
line = np.zeros(30)
door_position = [9, 15, 25]
for dp in door_position:
  line[dp] = 1

In [3]:
line

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [27]:
class point:
  def __init__(self):
    self.position = random.randint(0,20)
    self.noise = 0
 
  def set(self, position):
    self.position = position
 
  def move(self, dist):
    self.position = (self.position + dist)%len(line)
  
  def set_noise(self, noise):
    self.noise = noise
 
  def get_reading(self):
    rtn = []
    for door in door_position:
      rtn.append(abs(door - self.position))
    return rtn
  
  def gaussian(self, mu, sigma, x):
    return float(exp(- ((mu - x) ** 2) / (sigma ** 2) / 2.0) / sqrt(2.0 * pi * (sigma ** 2)))
  
  def measurement_prob(self, measurement):
    prob = 1.0
    reading = self.get_reading()
    for i in range(3):
      prob *= self.gaussian(reading[i], self.noise, measurement[i])
    return prob
  
  def __repr__(self):
    return '[position='+str(self.position)+', reading='+str(self.get_reading())+']'

In [5]:
N = 10
p = []
 
for _ in range(N):
  x = point()
  x.set_noise(0.5)
  p.append(x)
p

[[position=11, reading=[2, 4, 14]],
 [position=16, reading=[7, 1, 9]],
 [position=16, reading=[7, 1, 9]],
 [position=10, reading=[1, 5, 15]],
 [position=19, reading=[10, 4, 6]],
 [position=15, reading=[6, 0, 10]],
 [position=19, reading=[10, 4, 6]],
 [position=13, reading=[4, 2, 12]],
 [position=18, reading=[9, 3, 7]],
 [position=16, reading=[7, 1, 9]]]

In [6]:
actual_point = point()

actual_point.set(5)
actual_point.set_noise(0.5)
Z = actual_point.get_reading()

In [38]:
N = 100
T = 10
 
# random points
p = []
 
for _ in range(N):
  x = point()
  x.set_noise(0.5)
  p.append(x)
#print(p[i].measurement_prob(1))
 
for rd in range(T):
  actual_point.move(2)
  Z = actual_point.get_reading()
  #print(actual_point)
  
  p2 = []
  for i in range(N):
    p[i].move(2)
    p2.append(p[i])
  p = p2
 
  # weight
  w = []
  for poi in p:
    prob = poi.measurement_prob(Z)
    w.append(prob)
  #print(w)
 
  #resampling
  p3 = []
  #normal weights
  norm_w = np.true_divide(w, np.sum(w, dtype='float'), where=(w!=0), dtype='float')
  #print(np.sum(w))
  #print(norm_w)
  index = [np.random.choice(np.arange(0, N), p = norm_w) for i in range(N)]
 
  for i in index:
    p3.append(p[i])
 
  for i in range(10):
    p3[i] = point()
    p3[i].set_noise(0.5)
  p = p3
 
  output = np.zeros(len(line))
  for i in p:
    output[i.position] += 1
  print('actual potition:',actual_point.position)
  print('best estimate:',output.tolist().index(max(output)))

actual potition: 1
best estimate: 2
actual potition: 3
best estimate: 2
actual potition: 5
best estimate: 2
actual potition: 7
best estimate: 7
actual potition: 9
best estimate: 8
actual potition: 11
best estimate: 9
actual potition: 13
best estimate: 13
actual potition: 15
best estimate: 15
actual potition: 17
best estimate: 17
actual potition: 19
best estimate: 19
